# Tuning Neural Networks with Normalization - Lab

## Introduction

For this lab on initialization and optimization, you'll build a neural network to perform a regression task.

It is worth noting that getting regression to work with neural networks can be difficult because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$, and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case for tinkering with normalization and optimization strategies to ensure proper convergence!

## Objectives
You will be able to:
* Build a neural network using Keras
* Normalize your data to assist algorithm convergence
* Implement and observe the impact of various initialization techniques

In [328]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras import layers
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from keras import optimizers
from sklearn.model_selection import train_test_split

## Loading the data

The data we'll be working with is data related to Facebook posts published during the year of 2014 on the Facebook page of a renowned cosmetics brand.  It includes 7 features known prior to post publication, and 12 features for evaluating the post impact. What we want to do is make a predictor for the number of "likes" for a post, taking into account the 7 features prior to posting.

First, let's import the data set, `dataset_Facebook.csv`, and delete any rows with missing data. Afterwards, briefly preview the data.

In [329]:
#Your code here; load the dataset and drop rows with missing values. Then preview the data.

df=pd.read_csv('dataset_Facebook.csv', delimiter=";")
df.head()


,Page total likes,Type,Category,Post Month,Post Weekday,Post Hour,Paid,Lifetime Post Total Reach,Lifetime Post Total Impressions,Lifetime Engaged Users,Lifetime Post Consumers,Lifetime Post Consumptions,Lifetime Post Impressions by people who have liked your Page,Lifetime Post reach by people who like your Page,Lifetime People who have liked your Page and engaged with your post,comment,like,share,Total Interactions
0,139441,Photo,2,12,4,3,0.0,2752,5091,178,109,159,3078,1640,119,4,79.0,17.0,100
1,139441,Status,2,12,3,10,0.0,10460,19057,1457,1361,1674,11710,6112,1108,5,130.0,29.0,164
2,139441,Photo,3,12,3,3,0.0,2413,4373,177,113,154,2812,1503,132,0,66.0,14.0,80
3,139441,Photo,2,12,2,10,1.0,50128,87991,2211,790,1119,61027,32048,1386,58,1572.0,147.0,1777
4,139441,Photo,2,12,2,3,0.0,7244,13594,671,410,580,6228,3200,396,19,325.0,49.0,393


In [330]:
df.isna().sum()

Page total likes                                                       0
Type                                                                   0
Category                                                               0
Post Month                                                             0
Post Weekday                                                           0
Post Hour                                                              0
Paid                                                                   1
Lifetime Post Total Reach                                              0
Lifetime Post Total Impressions                                        0
Lifetime Engaged Users                                                 0
Lifetime Post Consumers                                                0
Lifetime Post Consumptions                                             0
Lifetime Post Impressions by people who have liked your Page           0
Lifetime Post reach by people who like your Page   

In [331]:
df=df.dropna()

In [332]:
df.isna().sum().sum()

0

In [333]:
df.shape

(495, 19)

In [334]:
def visualize_training_results(results):
    history = results.history
    plt.figure()
    plt.plot(history['val_loss'])
    plt.plot(history['loss'])
    plt.legend(['val_loss', 'loss'])
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.show()
    
    plt.figure()
    plt.plot(history['val_mse'])
    plt.plot(history['mse'])
    plt.legend(['val_accuracy', 'accuracy'])
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.show()

## Defining the Problem

Define X and Y and perform a train-validation-test split.

X will be:
* Page total likes
* Post Month
* Post Weekday
* Post Hour
* Paid
along with dummy variables for:
* Type
* Category

Y will be the `like` column.

In [335]:
#Your code here; define the problem

y=df.like
X=df[['Page total likes',
'Post Month',
'Post Weekday',
'Post Hour',
'Paid',
'Type',
'Category']]
X.Type.value_counts()

Photo     421
Status     45
Link       22
Video       7
Name: Type, dtype: int64

In [336]:
dummy_X1= pd.get_dummies(X.Type, drop_first=True)
dummy_X1.astype('float')

,Photo,Status,Video
0,1.0,0.0,0.0
1,0.0,1.0,0.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,1.0,0.0,0.0
...,...,...,...
494,1.0,0.0,0.0
495,1.0,0.0,0.0
496,1.0,0.0,0.0
497,1.0,0.0,0.0


In [337]:
X=pd.concat([X, dummy_X1], axis=1)
X=X.drop('Type', axis=1)

In [338]:
X_train, X_test_val, y_train, y_test_val = train_test_split(X, y, test_size=0.2, random_state=42)

X_val, X_test, y_val, y_test=train_test_split(X_test_val, y_test_val, test_size=0.5, random_state=42)

In [339]:
X_train.shape

(396, 9)

## Building a Baseline Model

Next, build a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model.

In [340]:
#Simply run this code block, later you'll modify this model to tune the performance
np.random.seed(123)
model = Sequential()
model.add(layers.Dense(8, input_dim=9, activation='relu'))
model.add(layers.Dense(1, activation = 'linear'))

model.compile(optimizer= "sgd" ,loss='mse',metrics=['mse'])
hist = model.fit(X_train, y_train, batch_size=32, 
                 epochs=100, validation_data = (X_val, y_val), verbose=0)

### Evaluating the Baseline

Evaluate the baseline model for the training and validation sets.

In [341]:
#Your code here; evaluate the model with MSE

pred_train = model.predict(X_train).reshape(-1)
pred_train

array([178.97789, 178.97789, 178.97789, 178.97789, 178.97789, 178.97789,
       178.97789, 178.97789, 178.97789, 178.97789, 178.97789, 178.97789,
       178.97789, 178.97789, 178.97789, 178.97789, 178.97789, 178.97789,
       178.97789, 178.97789, 178.97789, 178.97789, 178.97789, 178.97789,
       178.97789, 178.97789, 178.97789, 178.97789, 178.97789, 178.97789,
       178.97789, 178.97789, 178.97789, 178.97789, 178.97789, 178.97789,
       178.97789, 178.97789, 178.97789, 178.97789, 178.97789, 178.97789,
       178.97789, 178.97789, 178.97789, 178.97789, 178.97789, 178.97789,
       178.97789, 178.97789, 178.97789, 178.97789, 178.97789, 178.97789,
       178.97789, 178.97789, 178.97789, 178.97789, 178.97789, 178.97789,
       178.97789, 178.97789, 178.97789, 178.97789, 178.97789, 178.97789,
       178.97789, 178.97789, 178.97789, 178.97789, 178.97789, 178.97789,
       178.97789, 178.97789, 178.97789, 178.97789, 178.97789, 178.97789,
       178.97789, 178.97789, 178.97789, 178.97789, 

In [342]:
#Your code here; inspect the loss function through the history object

> Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.

## Normalize the Input Data

Normalize the input features by subtracting each feature mean and dividing by the standard deviation in order to transform each into a standard normal distribution. Then recreate the train-validate-test sets with the transformed input data.

In [343]:
## standardize/categorize

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)


In [344]:
X_train, X_test_val, y_train, y_test_val = train_test_split(X, y, test_size=0.2, random_state=42)

X_val, X_test, y_val, y_test=train_test_split(X_test_val, y_test_val, test_size=0.5, random_state=42)

## Refit the Model and Reevaluate

Great! Now refit the model and once again assess it's performance on the training and validation sets.

In [345]:
#Your code here; refit a model as shown above
np.random.seed(123)
model = Sequential()
model.add(layers.Dense(8, input_dim=9, activation='relu'))
model.add(layers.Dense(1, activation = 'linear'))

model.compile(optimizer= "sgd" ,loss='mse',metrics=['mse'])
hist = model.fit(X_train, y_train, batch_size=32, 
                 epochs=100, validation_data = (X_val, y_val), verbose=0)

In [346]:
#Rexamine the loss function
pred_train = model.predict(X_train).reshape(-1)
pred_train

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, na

> Note that you still haven't achieved convergence! From here, it's time to normalize the output data.

## Normalizing the output

Normalize Y as you did X by subtracting the mean and dividing by the standard deviation. Then, resplit the data into training and validation sets as we demonstrated above, and retrain a new model using your normalized X and Y data.

In [347]:
#Your code here: redefine Y after normalizing the data.
y=(y-np.mean(y))/np.std(y) 


In [348]:
#Your code here; create training and validation sets as before. Use random seed 123.
X_train, X_test_val, y_train, y_test_val = train_test_split(X, y, test_size=0.2, random_state=42)

X_val, X_test, y_val, y_test=train_test_split(X_test_val, y_test_val, test_size=0.5, random_state=42)

In [349]:
#Your code here; rebuild a simple model using a relu layer followed by a linear layer. (See our code snippet above!)
np.random.seed(123)
model = Sequential()
model.add(layers.Dense(8, input_dim=9, activation='relu'))
model.add(layers.Dense(1, activation = 'linear'))

model.compile(optimizer= "sgd" ,loss='mse',metrics=['mse'])
hist = model.fit(X_train, y_train, batch_size=32, 
                 epochs=100, validation_data = (X_val, y_val), verbose=0)

Again, reevaluate the updated model.

In [350]:
#Your code here; MSE
pred_train = model.predict(X_train).reshape(-1)
pred_val = model.predict(X_val).reshape(-1)  

MSE_train = np.mean((pred_train-y_train)**2)
MSE_val = np.mean((pred_val-y_val)**2)

print("MSE_train:", MSE_train)
print("MSE_val:", MSE_val)

MSE_train: 1.0024408898572925
MSE_val: 0.49746430171971223


In [351]:
#Your code here; loss function
pred_train = model.predict(X_train).reshape(-1)
pred_train

array([-0.23528203, -0.23340225,  0.08265512,  0.53356767, -0.21890354,
       -0.19733131,  0.1075437 , -0.30267996,  0.7053416 , -0.00102524,
        0.19557935, -0.10285539,  0.05825097,  0.05228304, -0.23351583,
       -0.23637554, -0.21567261, -0.2561264 , -0.15360636, -0.20658347,
       -0.04200689, -0.29624757, -0.11093538, -0.22783542,  0.15173909,
        0.43707597, -0.08424101, -0.05613197, -0.41973808, -0.06148822,
       -0.21833181,  0.09090181, -0.06313281,  0.12811321, -0.13789287,
        0.41453487,  0.48983082, -0.37004694, -0.19307512, -0.07508244,
       -0.07234821, -0.03574724, -0.33634725,  0.11995288,  0.0544626 ,
        0.09241364, -0.12505099,  0.10201644, -0.34468842, -0.3508745 ,
        0.16824323, -0.3635187 , -0.3344897 ,  0.613908  ,  0.00148015,
       -0.20412728,  0.2863049 , -0.21433547,  0.09908298, -0.1694763 ,
        0.5287459 ,  0.05225857,  0.0952199 , -0.09093434,  0.2400074 ,
       -0.2576403 ,  0.01315289, -0.5160421 ,  0.01848959, -0.03

Great! Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.)

## Using Weight Initializers

Below, take a look at the code provided to see how to modify the neural network to use alternative initialization and optimization strategies. At the end, you'll then be asked to select the model which you believe is the strongest.

In [352]:
Y_train=y_train
Y_val=y_val
Y_test=y_test

##  He Initialization

In [353]:
np.random.seed(123)
model = Sequential()
model.add(layers.Dense(8, input_dim=9, kernel_initializer= "he_normal",
                activation='relu'))
model.add(layers.Dense(1, activation = 'linear'))

model.compile(optimizer= "sgd" ,loss='mse',metrics=['mse'])
hist = model.fit(X_train, Y_train, batch_size=32, 
                 epochs=100, validation_data = (X_val, Y_val),verbose=0)

In [354]:
pred_train = model.predict(X_train).reshape(-1)
pred_val = model.predict(X_val).reshape(-1)

MSE_train = np.mean((pred_train-Y_train)**2)
MSE_val = np.mean((pred_val-Y_val)**2)

In [355]:
print(MSE_train)
print(MSE_val)

0.9792943675410861
0.4909449996629464


## Lecun Initialization

In [356]:
np.random.seed(123)
model = Sequential()
model.add(layers.Dense(8, input_dim=9, 
                kernel_initializer= "lecun_normal", activation='tanh'))
model.add(layers.Dense(1, activation = 'linear'))

model.compile(optimizer= "sgd" ,loss='mse',metrics=['mse'])
hist = model.fit(X_train, Y_train, batch_size=32, 
                 epochs=100, validation_data = (X_val, Y_val), verbose=0)

In [357]:
pred_train = model.predict(X_train).reshape(-1)
pred_val = model.predict(X_val).reshape(-1)

MSE_train = np.mean((pred_train-Y_train)**2)
MSE_val = np.mean((pred_val-Y_val)**2)

In [358]:
print(MSE_train)
print(MSE_val)

1.0304396047372788
0.4922410001698343


Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

## RMSprop

In [359]:
np.random.seed(123)
model = Sequential()
model.add(layers.Dense(8, input_dim=9, activation='relu'))
model.add(layers.Dense(1, activation = 'linear'))

model.compile(optimizer= "rmsprop" ,loss='mse',metrics=['mse'])
hist = model.fit(X_train, Y_train, batch_size=32, 
                 epochs=100, validation_data = (X_val, Y_val), verbose = 0)

In [360]:
pred_train = model.predict(X_train).reshape(-1)
pred_val = model.predict(X_val).reshape(-1)

MSE_train = np.mean((pred_train-Y_train)**2)
MSE_val = np.mean((pred_val-Y_val)**2)

In [361]:
print(MSE_train)
print(MSE_val)

1.01635893620607
0.47849355442735303


## Adam

In [362]:



np.random.seed(123)
model = Sequential()
model.add(layers.Dense(8, input_dim=9, activation='relu'))
model.add(layers.Dense(1, activation = 'linear'))

model.compile(optimizer= "Adam" ,loss='mse',metrics=['mse'])
hist = model.fit(X_train, Y_train, batch_size=32, 
                 epochs=100, validation_data = (X_val, Y_val), verbose = 0)

In [363]:
pred_train = model.predict(X_train).reshape(-1)
pred_val = model.predict(X_val).reshape(-1)

MSE_train = np.mean((pred_train-Y_train)**2)
MSE_val = np.mean((pred_val-Y_val)**2)

In [364]:
print(MSE_train)
print(MSE_val)

1.0115725151943056
0.47882715294456224


## Learning Rate Decay with Momentum


In [372]:
np.random.seed(123)
sgd = optimizers.SGD(lr=0.03, decay=0.0001, momentum=0.9)
model = Sequential()
model.add(layers.Dense(8, input_dim=9, activation='relu'))
model.add(layers.Dense(1, activation = 'linear'))

model.compile(optimizer= 'sgd' ,loss='mse',
              metrics=['mse'])
hist = model.fit(X_train, Y_train, batch_size=32, 
                 epochs=100, validation_data = (X_val, Y_val), verbose = 0)

In [373]:
hist.history.keys()

dict_keys(['val_loss', 'val_mse', 'loss', 'mse'])

In [374]:
pred_train = model.predict(X_train).reshape(-1)
pred_val = model.predict(X_val).reshape(-1)

MSE_train = np.mean((pred_train-Y_train)**2)
MSE_val = np.mean((pred_val-Y_val)**2)

In [375]:
print(MSE_train)
print(MSE_val)

1.0470611419198814
0.496269550008077


## Selecting a Final Model

Now, select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [376]:
#Your code here
pred_test = model.predict(X_test).reshape(-1)

MSE_train = np.mean((pred_train-Y_train)**2)
MSE_test = np.mean((pred_test-Y_test)**2)
print(MSE_train)
print(MSE_val)
print(MSE_test)

1.0470611419198814
0.496269550008077
0.7812355241151566


In [ ]:
import matplotlib.pyplot as plt


visualize_training_results(hist)

## Summary  

In this lab, you worked to ensure your model converged properly. Additionally, you also investigated the impact of varying initialization and optimization routines.